In [ ]:
%matplotlib inline
import logging
logging.basicConfig(level=logging.CRITICAL)

import os
import sys
sys.path.append('..')

from scipy.misc import imread

from util.config import load_config
from nnet import predict
from util import visualize
from dataset.pose_dataset import data_to_input


cfg = load_config("evaluate_pose_cfg.yaml")

# Load and setup CNN part detector
sess, inputs, outputs = predict.setup_pose_prediction(cfg)

In [ ]:
import pathlib
import random
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import itertools
import utils

reconstruction_repo_path = pathlib.Path.home() / 'dev/Reconstruction'
key_point_regressor = utils.TFLiteKeyPointRegressor(reconstruction_repo_path / 'komb/resources/moby/point_regressor.tflite')

dataset_path = pathlib.Path('/moby/datasets/deepercut/volumental/cropped')
image_group_path = dataset_path / 'group_9'
annotations_path = dataset_path / 'annotations'

image_paths = list(image_group_path.iterdir())

In [ ]:
random.shuffle(image_paths)
image_paths_to_plot = image_paths[:8]

fig, axes = plt.subplots(len(image_paths_to_plot) // 4, 4, figsize=(40, 10 * len(image_paths_to_plot) // 4), squeeze=True)
axes = itertools.chain(*axes)

for image_path, ax in zip(image_paths_to_plot, axes):
    image = imread(image_path, mode='RGB')

    image_batch = data_to_input(image)

    # Compute prediction with the CNN
    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
    scmap, locref, _ = predict.extract_cnn_output(outputs_np, cfg)

    # Extract maximum scoring location from the heatmap, assume 1 person
    pose = predict.argmax_pose_predict(scmap, locref, cfg.stride)
    
    keypoint_path = utils.keypoint_path_from_cropped_image_path(image_path)
    try:
        with open(keypoint_path) as keypoint_file:
            keypoints = json.load(keypoint_file)
    except FileNotFoundError:
        print(json_path)
        continue
    keypoints = np.array([[coords[0], coords[1]] for keypoint, coords in keypoints.items()])
    
    mu = utils.predict_with_regressor(image, key_point_regressor)
    
    ax.imshow(image)
    ax.scatter(mu[:, 0], mu[:, 1], c='blue')
    ax.scatter(pose[:,0], pose[:,1], c='orange')
    ax.scatter(keypoints[:,0], keypoints[:,1], c='red')

In [ ]:
for image_path, _ in zip(image_paths_to_plot, range(4)):
    image = imread(image_path, mode='RGB')

    image_batch = data_to_input(image)

    # Compute prediction with the CNN
    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
    scmap, locref, _ = predict.extract_cnn_output(outputs_np, cfg)

    # Extract maximum scoring location from the heatmap, assume 1 person
    pose = predict.argmax_pose_predict(scmap, locref, cfg.stride)
    
    # Visualise
    visualize.show_heatmaps(cfg, image, scmap, pose)

In [ ]:
import itertools
import pandas as pd

path = pathlib.Path('/moby/datasets/real_feet_resized/Feet Annotation Batch 1 tagging results dwHQG4wrvC9MeWxTt/')

df = pd.read_csv('/moby/datasets/real_feet_resized/Feet Annotation Batch 1 tagging results dwHQG4wrvC9MeWxTt/structure.csv')
df['ImagePath'] = df["Filename"].apply(lambda s: str(path /'images'/ s))

In [ ]:
df = df.sample(frac=1)
df_sampled = df[:20]

fig, axes = plt.subplots(len(df_sampled) // 4, 4, figsize=(40, 10 * len(df_sampled) // 4), squeeze=True)
axes = itertools.chain(*axes)
padding=10
for (index, row), ax in zip(df_sampled.iterrows(), axes):
    image = imread(row['ImagePath'], mode='RGB')
    image = image[row['Top']-padding:row['Top']+row['Height']+padding, row['Left']-padding:row['Left']+row['Width']+padding]
    image_batch = data_to_input(image)

    # Compute prediction with the CNN
    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
    scmap, locref, _ = predict.extract_cnn_output(outputs_np, cfg)

    # Extract maximum scoring location from the heatmap, assume 1 person
    pose = predict.argmax_pose_predict(scmap, locref, cfg.stride)
    
    mu = utils.predict_with_regressor(image, key_point_regressor)

    ax.imshow(image)
    ax.scatter(pose[:,0], pose[:,1], c='orange')    
    ax.scatter(mu[:, 0], mu[:, 1], c='blue')

In [ ]:
for (index, row), _ in zip(df_sampled.iterrows(), range(4)):
    image = imread(row['ImagePath'], mode='RGB')
    image = image[row['Top']-padding:row['Top']+row['Height']+padding, row['Left']-padding:row['Left']+row['Width']+padding]
    image_batch = data_to_input(image)

    # Compute prediction with the CNN
    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
    scmap, locref, _ = predict.extract_cnn_output(outputs_np, cfg)

    # Extract maximum scoring location from the heatmap, assume 1 person
    pose = predict.argmax_pose_predict(scmap, locref, cfg.stride)
    
    visualize.show_heatmaps(cfg, image, scmap, pose)

In [ ]:
import tqdm
deepercut_error_pixel = 0
regressor_error_pixel = 0
deepercut_error_relative = 0
regressor_error_relative = 0
num_images = 0
for image_path in tqdm.tqdm(image_paths):
    image = imread(image_path, mode='RGB')

    image_batch = data_to_input(image)

    # Compute prediction with the CNN
    outputs_np = sess.run(outputs, feed_dict={inputs: image_batch})
    scmap, locref, _ = predict.extract_cnn_output(outputs_np, cfg)

    # Extract maximum scoring location from the heatmap, assume 1 person
    pose = predict.argmax_pose_predict(scmap, locref, cfg.stride)
    
    mu = utils.predict_with_regressor(image, key_point_regressor)
    
    keypoint_path = utils.keypoint_path_from_cropped_image_path(image_path)
    try:
        with open(keypoint_path) as keypoint_file:
            keypoints = json.load(keypoint_file)
    except FileNotFoundError:
        print(json_path)
        continue
    keypoints = np.array([[coords[0], coords[1]] for keypoint, coords in keypoints.items()])
    deepercut_error_pixel += np.mean(np.linalg.norm(pose[:, :2] - keypoints, axis=1))
    regressor_error_pixel += np.mean(np.linalg.norm(mu - keypoints, axis=1))
    
    keypoints = np.divide(keypoints, image.shape[:2])
    pose = np.divide(pose[:, :2], image.shape[:2])
    mu = np.divide(mu, image.shape[:2])
    
    deepercut_error_relative += np.mean(np.linalg.norm(pose - keypoints, axis=1))
    regressor_error_relative += np.mean(np.linalg.norm(mu - keypoints, axis=1))
    
    num_images += 1
    
print('DeeperCut pixel:', deepercut_error_pixel / num_images)
print('Regressor pixle:', regressor_error_pixel / num_images)
print('DeeperCut relative:', deepercut_error_relative / num_images)
print('Regressor relative:', regressor_error_relative / num_images)